This is a quick illustration of how to use the code in this repository. 

In [1]:
import os
os.chdir("../") # change directory to parent directory
print("Working directory: ", os.getcwd())

Working directory:  /Users/almarahat/Dropbox/repo/EHVI_Gauss-Hermite


In [2]:
import sys
sys.path.append("./src/") # add code directory to path

In [3]:
import pandas as pd
import timeit
from IPython.display import clear_output
import import_ipynb
from gauss_hermite import mGauss_hermite
from ehvi_wrapper import generate_txt_input_pf, run_ehvi
from utils import *
import numpy as np
import warnings
import matplotlib.pyplot as plt
from evoalgos.performance import FonsecaHyperVolume as FH
from scipy.stats import multivariate_normal as MVN

source = "data/pf/" # directory where Pareto front data is saved
# load data from csv
probd2_2d = pd.read_csv(source + "DTLZ2.2D.pf", header=None, delimiter=" ", dtype=float)
pfd2_2d = probd2_2d.to_numpy()

In [4]:
# set seed for repeatability
np.random.seed(1234)

In [5]:
%%time
ndim = 2 # number of dimensions of the objectvie space
# code base for the exact calculation of EHVI
exact_code_base = "EHVI/EHVI_2D/" # use "EHVI/EHVI_3D/" for ndim=3
# lower and upper bounds
lb = np.zeros(ndim)
ub = np.ones(ndim)
# example mean and covariance matrix
mean = get_mean(lb, ub)
cov = get_cov_indep(lb, ub) # This creates an uncorrelated covariance matrix.
                            # for a correlated covaraince matrix call "get_cov(lb,ub)"
print("The exmaple distribution is deifned by the mean: ", mean, " and covariance matrix: ")
print(cov)
prune = 0.2 # pruning paramter between 0 and 1. 
nmc = 10000 # number of monte carlo samples
ngh = np.arange(3, 16, 1) # number of points per dimensions for Gauss-Hermite experiments, 
                          # we vary then between 3 and 15
# current hpv
ref_vect = np.ones(ndim)*11 # this is for DTLZ2-2D
print("The reference vector used here is: ", ref_vect)
hpv = FH(ref_vect)
current_hv = hpv.assess_non_dom_front(pfd2_2d)
print("HPV of the current front: ", current_hv)
results, labels = run_exp(mean, cov, pfd2_2d, ref_vect, nmc, ngh, code_base=exact_code_base)
df = pd.DataFrame(data=results, index=labels)
print(df)

The exmaple distribution is deifned by the mean:  [0.19151945 0.62210877]  and covariance matrix: 
[[0.43772774 0.        ]
 [0.         0.78535858]]
The reference vector used here is:  [11. 11.]
HPV of the current front:  120.21062692728928
                  0
mean       0.191519
mean       0.622109
std        0.661610
std        0.886205
ref       11.000000
ref       11.000000
3          2.347278
4          3.041684
5          3.336036
6          3.190702
7          3.583626
8          3.328133
9          3.542656
10         3.383657
11         3.496399
12         3.408027
13         3.452649
14         3.416683
15         3.412840
mc_avg     3.357030
mc_std     4.948185
mc_err     0.049482
measured   3.402860
CPU times: user 5.77 s, sys: 41.3 ms, total: 5.81 s
Wall time: 5.8 s


### Key to the labels. 

- mean: mean of the Gaussian distribution.
- std: standard deviation of the Gaussian distribution. 
- ref: reference vector for hypervolume computation. 
- number: number of points per dimension for Gauss-Hermite approximation
- mc_avg: average MC approximation. 
- mc_std: standard deviation of the MC approximation. 
- mc_err: error estimation for MC. 
- measured: exact EHVI
